In [1]:
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

dirs = [".."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

from swift.llm import (
    get_model_tokenizer, get_template, inference, ModelType, get_default_template_type, TemplateType
)

from custom import CustomModelType, CustomTemplateType

# model_dir = '/home/hanlv/workspace/code/research/infodemic/LLM/swift/examples/pytorch/llm/output/openchat_3.5/with_solar_info/brave/data1/lr=1.1e-4/lora_rank=8/split_type=8:2-train_ratio=1.0-20240116-00:52:51/checkpoint-609'
model_type = CustomModelType.darebeagle_7b
template_type = get_default_template_type(model_type)
print(f"Template type: {template_type}")

model, tokenizer = get_model_tokenizer(model_type)
model.generation_config.max_new_tokens = 512
template = get_template(template_type, tokenizer)



2024-01-21 01:31:05,431 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-01-21 01:31:05,434 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-01-21 01:31:05,459 - modelscope - INFO - Loading done! Current index file version is 1.11.0, with md5 dc3cda2b56a86b0242cd145399a14071 and a total number of 953 components indexed
[INFO:swift] Setting torch_dtype: torch.float16
[INFO:swift] model_config: MistralConfig {
  "_name_or_path": "/home/css/models/DareBeagle-7B-v2",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,


Template type: neural


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
# template_type = TemplateType.chatml
query = "longest river?"

response, history = inference(
    model, template, query, 
    verbose=False, prompt_prefix='', output_prefix=''
)

print(response)


The longest river in the world is the Nile River, which is located in Africa and has a length of approximately 6,650 kilometers (4,130 miles).

The Nile River flows through 11 countries in Africa, including Tanzania, Uganda, Rwanda, Burundi, Democratic Republic of the Congo, Kenya, Ethiopia, Eritrea, Sudan, South Sudan, and Egypt. It is a vital source of water and sustains the livelihoods of millions of people in the region.

The Nile River is divided into two main tributaries: the White Nile and the Blue Nile. The White Nile originates in the Great Lakes region of Central Africa and flows northward through Uganda, South Sudan, and Sudan. The Blue Nile originates in Ethiopia and flows westward into Sudan, where it meets the White Nile near Khartoum. The combined flow of the two tributaries forms the main Nile River, which flows northward through Egypt and empties into the Mediterranean Sea.

The Nile River has played a significant role in the history and development of the civilization